In [ ]:
from urllib.request import urlopen
from xml.etree.ElementTree import parse

In [5]:
doc = parse(r'D:\1.xml')

In [6]:
root = doc.getroot()

In [7]:
root.remove(root.find('cr'))

In [ ]:
root.remove(root.find('sri'))

In [8]:
root.getchildren().index(root.find('nm'))

1

In [9]:
e = Element('spam')
e.text = 'this is the test'

In [10]:
root.insert(2,e)

In [11]:
doc.write(r'd:\2.xml',xml_declaration=True)